# Description
___
## Data
* CIFAR10의 32x32, 3채널
* train : 50000, test : 10000

---
## ResNet
* 본 코드에서는 Resnet Neural Network를 구성하는 Basic_Block과 BottleNeck_Block을 직접 만들었다.
* 그리고,Class ResNet의 Layer를 직접 구성하였다.
* 20 에포크 모델의 결과, Accuracy 88% ~ 90%까지 구할 수 있었다.
___
### *SpinalNet*
* 모델의 성능 개선을 위해서 SpinalNet을 출력 Layer로 추가 구성하였지만 별다른 차이가 없었다.
* 모델의 성능은 여전히  Accuracy 88% ~ 90%으로 이전 모델고 전혀 차이가 없었다.

# Library

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import torch.optim as optim
import os
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau


# device = "cuda" if torch.cuda.is_available else "cpu"
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print(device)

cuda


# Load Data

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])


# train_data
train_dataset = torchvision.datasets.CIFAR10('./data', train = True, download = True, transform = transforms.Compose([
                                                                                                          transforms.RandomHorizontalFlip(),
                                                                                                          transforms.ToTensor(),
                                                                                                          transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                                                                                                          ]))

# train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128,
                                          shuffle=True, num_workers=4)


# test_data
test_dataset = torchvision.datasets.CIFAR10('./data', train = False, transform = transforms.Compose([
                                                                                         transforms.ToTensor(),
                                                                                         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]))

# test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100,
                                         shuffle=False, num_workers=4)

Files already downloaded and verified


# Block

In [3]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        self.dropout = nn.Dropout(0.1)
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.dropout(self.bn1(self.conv1(x))))
        out = self.dropout(self.bn2(self.conv2(out)))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.dropout = nn.Dropout(0.1)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.dropout(self.bn1(self.conv1(x))))
        out = F.relu(self.dropout(self.bn2(self.conv2(out))))
        out = self.dropout(self.bn3(self.conv3(out)))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [4]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

        # Add Spinal Layers
        half_in_size = 512 * block.expansion // 2
        layer_width = 20
        self.fc_spinal_layer1 = nn.Sequential(
            nn.Linear(half_in_size, layer_width),
            nn.ReLU(inplace=True),
        )
        self.fc_spinal_layer2 = nn.Sequential(
            nn.Linear(half_in_size+layer_width, layer_width),
            nn.ReLU(inplace=True),
        )
        self.fc_spinal_layer3 = nn.Sequential(
            nn.Linear(half_in_size+layer_width, layer_width),
            nn.ReLU(inplace=True),
        )
        self.fc_spinal_layer4 = nn.Sequential(
            nn.Linear(half_in_size+layer_width, layer_width),
            nn.ReLU(inplace=True),
        )
        self.fc_out = nn.Sequential(
            nn.Linear(layer_width*4, num_classes),
        )

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        # Flatten the output of ResNet
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)

        # Pass through Spinal Layers
        half_in_size = out.size(1) // 2  # half of the flattened size
        x1 = self.fc_spinal_layer1(out[:, 0:half_in_size])
        x2 = self.fc_spinal_layer2(torch.cat([out[:, half_in_size:2*half_in_size], x1], dim=1))
        x3 = self.fc_spinal_layer3(torch.cat([out[:, 0:half_in_size], x2], dim=1))
        x4 = self.fc_spinal_layer4(torch.cat([out[:, half_in_size:2*half_in_size], x3], dim=1))

        # Concatenate the outputs of Spinal Layers
        out = torch.cat([x1, x2], dim=1)
        out = torch.cat([out, x3], dim=1)
        out = torch.cat([out, x4], dim=1)

        # Final output
        out = self.fc_out(out)

        return F.log_softmax(out,dim=-1)

def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())

In [5]:
# Importing Model
model = ResNet18().to(device)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# model training
def model_training(model, device, train_dataloader, optimizer, train_acc, train_losses):

    model.train()
    pbar = tqdm(train_dataloader)
    correct = 0
    processed = 0
    running_loss = 0.0

    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        y_pred = model(data)
        loss = F.cross_entropy(y_pred, target)


        train_losses.append(loss)
        loss.backward()
        optimizer.step()
        correct += y_pred.argmax(dim=1).eq(target).sum()
        processed += len(data)
        # print statistics
        running_loss += loss.item()
        pbar.set_description(desc=f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        train_acc.append(100*correct/processed)

def model_testing(model, device, test_dataloader, test_acc, test_losses, misclassified = []):

    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():

        for index, (data, target) in enumerate(test_dataloader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            test_loss += F.cross_entropy(output, target)
            correct += pred.eq(target.view_as(pred)).sum()
    test_loss /= len(test_dataloader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_dataloader.dataset),
        100. * correct / len(test_dataloader.dataset)))

    test_acc.append(100. * correct / len(test_dataloader.dataset))


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=True)
# scheduler = StepLR(optimizer, step_size=15, gamma=0.1)

train_acc = []
train_losses = []
test_acc = []
test_losses = []

EPOCHS = 20

for i in range(EPOCHS):
    print(f'EPOCHS : {i}')
    model_training(model, device, train_loader, optimizer, train_acc, train_losses)
    scheduler.step(train_losses[-1])
    model_testing(model, device, test_loader, test_acc, test_losses)

EPOCHS : 0


Loss=1.2529735565185547 Batch_id=390 Accuracy=37.09: 100%|██████████| 391/391 [00:33<00:00, 11.54it/s]



Test set: Average loss: 0.0141, Accuracy: 4901/10000 (49.01%)

EPOCHS : 1


Loss=1.0485663414001465 Batch_id=390 Accuracy=57.45: 100%|██████████| 391/391 [00:25<00:00, 15.11it/s]



Test set: Average loss: 0.0103, Accuracy: 6348/10000 (63.48%)

EPOCHS : 2


...


Test set: Average loss: 0.0043, Accuracy: 8832/10000 (88.32%)

EPOCHS : 18


Loss=0.07241184264421463 Batch_id=390 Accuracy=98.40: 100%|██████████| 391/391 [00:25<00:00, 15.12it/s] 



Test set: Average loss: 0.0044, Accuracy: 8837/10000 (88.37%)

EPOCHS : 19


Loss=0.04022303968667984 Batch_id=390 Accuracy=98.76: 100%|██████████| 391/391 [00:25<00:00, 15.26it/s] 



Test set: Average loss: 0.0045, Accuracy: 8866/10000 (88.66%)

